# 1. Introduction.
In this project I was working on a simplified dump of anonymized data from the travel forum on stackexchange https://travel.stackexchange.com/, which consists of 8 tables:
- Badges
- Comments
- PostHistory
- Post Links
- Posts
- Tags
- Users
- Votes

Each data frame consists of many columns, where some of them connects tables with each other by some key column. In this project I will use pandas built-in functions to recreate given SQL queries and pandas.read_sql_query() function in order to check my solution.

# 2. Loading data sets.

In [35]:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3
import tempfile

#READING CSV FILES INTO THE MEMORY
Badges = pd.read_csv("travel_simple_frames/Badges.csv.gz",compression="gzip")
Comments = pd.read_csv("travel_simple_frames/Comments.csv.gz",compression="gzip")
PostLinks = pd.read_csv("travel_simple_frames/PostLinks.csv.gz",compression="gzip")
Posts = pd.read_csv("travel_simple_frames/Posts.csv.gz",compression="gzip")
Tags = pd.read_csv("travel_simple_frames/Tags.csv.gz",compression="gzip")
Users = pd.read_csv("travel_simple_frames/Users.csv.gz",compression="gzip")
Votes = pd.read_csv("travel_simple_frames/Votes.csv.gz",compression="gzip")

In the next step I established a connection with a temporal database in order to be able to execute reference solution.

In [36]:
#INITIALIZING TEMPORTAL DATABASE
db = os.path.join(tempfile.mkdtemp(), 'temporaldb.db')

#If file already exists, we delete it
if os.path.isfile(db):
    os.remove(db)
#Connect to database
conn = sqlite3.connect(db)

#Import tables
Badges.to_sql("Badges",conn)
Comments.to_sql("Comments",conn)
PostLinks.to_sql("PostLinks",conn)
Posts.to_sql("Posts",conn)
Tags.to_sql("Tags",conn)
Users.to_sql("Users",conn)
Votes.to_sql("Votes",conn)

# 3. SQL queries solutions.
## 3.1. First query.

### Reference solution:

In [37]:
first_sql = pd.read_sql_query("""
                  SELECT Posts.Title, RelatedTab.NumLinks
                  FROM
                      (SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
                       FROM PostLinks
                       GROUP BY RelatedPostId) AS RelatedTab
                  JOIN Posts ON RelatedTab.PostId=Posts.Id
                  WHERE Posts.PostTypeId=1
                  ORDER BY NumLinks DESC
                  """,conn)

### Pandas functions solutions:

In [38]:
#At first I execute inner query by finding the count(*) and selecting PostId
Related_Tab = PostLinks.groupby("RelatedPostId").size().to_frame('NumLinks') \
.reset_index().rename(columns={'RelatedPostId':'PostId'})

#Next I filter Posts table and do the join with Related_Tab
Posts_where = Posts[Posts['PostTypeId']==1]
join_query = pd.merge(Related_Tab, Posts_where, left_on='PostId',right_on='Id')

#Lastly I select required columns and order rows by NumLinks using mergesort to
#preserve row order
select_query = join_query[['Title','NumLinks']]
order_query = select_query.sort_values(by='NumLinks', 
                        kind='mergesort', ascending = False) \
                        .reset_index(drop=True)

The solution query result looks as follows:

In [39]:
order_query.head(6)

,Title,NumLinks
0,Is there a way to find out if I need a transit...,594
1,Do I need a visa to transit (or layover) in th...,585
2,Should my first trip be to the country which i...,331
3,Should I submit bank statements when applying ...,259
4,How much electronics and other valuables can I...,197
5,How does the Schengen 90/180 rule work?,192


Lastly I compared my solution with the reference one and the obtained result is positive:

In [40]:
first_sql.equals(order_query)

True

## 3.2. Second query.
### Reference solution:

In [41]:
second_sql = pd.read_sql_query("""
                               SELECT
                                   Users.DisplayName,
                                   Users.Age,
                                   Users.Location,
                                   SUM(Posts.FavoriteCount) AS FavoriteTotal,
                                   Posts.Title AS MostFavoriteQuestion,
                                   MAX(Posts.FavoriteCount) 
                                       AS MostFavoriteQuestionLikes
                                   FROM Posts
                                   JOIN Users ON Users.Id=Posts.OwnerUserId
                                   WHERE Posts.PostTypeId=1
                                   GROUP BY OwnerUserId
                                   ORDER BY FavoriteTotal DESC
                                   LIMIT 10
                               """,conn)

### Pandas functions solutions:

In [42]:
#I start with filtering the Posts table and joining to it Users table
Posts_where1 = Posts[Posts['PostTypeId']==1]
join_query1 = pd.merge(Users,Posts_where1,
                       left_on='Id', 
                       right_on='OwnerUserId')

#Then I assign a new column - 'FavoriteCount' which is a sum of OwnerUserId entries
join_query1['FavoriteTotal'] = join_query1.groupby('OwnerUserId')['FavoriteCount'] \
                        .transform('sum')

#Next I group the table by OwnerUserId and calculate max in every group while
#storing the indices with FavoriteCount = max of a group
max_query = join_query1.groupby(['OwnerUserId'])['FavoriteCount'] \
                        .transform('max') == join_query1['FavoriteCount']

#Lastly I rename the column, select them, sort the values by FavoriteTotal and lastly
#take the top 10 values in a dataframe
order_query1 = join_query1[max_query].rename(
    columns={'Title':'MostFavoriteQuestion','FavoriteCount':'MostFavoriteQuestionLikes'})
order_query1 = order_query1[['DisplayName','Age','Location','FavoriteTotal',
                              'MostFavoriteQuestion','MostFavoriteQuestionLikes']]
order_query1 = order_query1.sort_values('FavoriteTotal',ascending=False) \
                            .head(10).reset_index(drop=True)

The solution query result looks as follows:

In [43]:
order_query1

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,NaN,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,NaN,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,NaN,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


Lastly I compared my solution with the reference one and the obtained result is positive:

In [44]:
second_sql.equals(order_query1)

True

# 3.3. Third query.
### Reference solution:

In [45]:
third_sql = pd.read_sql_query("""
                                SELECT
                                    Posts.Title,
                                    CmtTotScr.CommentsTotalScore
                                FROM (
                                    SELECT
                                        PostID,
                                        UserID,
                                        SUM(Score) AS CommentsTotalScore
                                    FROM Comments
                                    GROUP BY PostID, UserID
                                ) AS CmtTotScr
                                JOIN Posts ON Posts.ID=CmtTotScr.PostID AND 
                                Posts.OwnerUserId=CmtTotScr.UserID
                                WHERE Posts.PostTypeId=1
                                ORDER BY CmtTotScr.CommentsTotalScore DESC
                                LIMIT 10
                                """,conn)

### Pandas functions solutions

In [46]:
#I start with executing the inner query. I group data by PostId and UserId and
#calculate a sum of Scores. Next I just rename the column to proper alias
CmtTotScr = Comments.groupby(['PostId','UserId'])['Score'].sum().reset_index()
CmtTotScr = CmtTotScr.rename(columns={'Score':'CommentsTotalScore'})

#Subsequently I filter Posts table and join it with the CmtTotScr query
#In the end I simply sort the table by CommentsTotalScore using mergesort to 
#preserve order of rows and select top 10 values 
Posts_where = Posts[Posts['PostTypeId']==1]
join_query = pd.merge(Posts_where,CmtTotScr,left_on=['Id','OwnerUserId'],
                      right_on=['PostId','UserId'])
select_query = join_query[['Title','CommentsTotalScore']].sort_values('CommentsTotalScore',
                        ascending = False, kind='mergesort').head(10).reset_index(drop=True)


The solution query result looks as follows:

In [47]:
select_query

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


Lastly I compared my solution with the reference one and the obtained result is positive:

In [48]:
third_query.equals(select_query)

True

In the end I close the connection to my temportal database by calling a function:

In [49]:
conn.close()